In [1]:
import requests
import math


In [2]:
def round_up_nearest_cent(number):
  """
  Rounds a number up to the nearest hundredth.
  """
  return math.ceil(number * 100) / 100

def get_orderbook_url(market_ticker):
    return f"https://api.elections.kalshi.com/trade-api/v2/markets/{market_ticker}/orderbook"

def get_all_markets(series_ticker):
    """Fetch all markets for a series, handling pagination"""
    all_markets = []
    cursor = None
    base_url = "https://api.elections.kalshi.com/trade-api/v2/markets"

    while True:
        # Build URL with cursor if we have one
        url = f"{base_url}?series_ticker={series_ticker}&limit=100"
        if cursor:
            url += f"&cursor={cursor}"

        response = requests.get(url)
        data = response.json()

        # Add markets from this page
        all_markets.extend(data['markets'])

        # Check if there are more pages
        cursor = data.get('cursor')
        if not cursor:
            break

        print(f"Fetched {len(data['markets'])} markets, total: {len(all_markets)}")
    return all_markets

def get_prices(orderbook_url):
  orderbook_response = requests.get(orderbook_url)
  orderbook_data = orderbook_response.json()

  highest_bid = orderbook_data['orderbook']['yes'][-1][0]
  highest_bid_volume = orderbook_data['orderbook']['yes'][-1][1]

  lowest_ask = 100 - orderbook_data['orderbook']['no'][-1][0]
  lowest_ask_volume = orderbook_data['orderbook']['no'][-1][1]

  return highest_bid, highest_bid_volume, lowest_ask, lowest_ask_volume

def get_prices_from_market(market):
    market_ticker = market['ticker']
    orderbook_url = get_orderbook_url(market_ticker)
    yes_bid, yes_bid_volume, yes_ask, yes_ask_volume = get_prices(orderbook_url)
    print(f"- {market['ticker'].split('-')[-1]}: Yes Bid: {yes_bid}¢ | Volume: {yes_bid_volume}, Yes Ask: {yes_ask}¢ | Volume: {yes_ask_volume}")
    return yes_bid, yes_bid_volume, yes_ask, yes_ask_volume

# Calculates the fair price to buy/sell a given number of contracts at the lowest ask/highest bid, including fees.
# Returns (taker_price, maker_price, taker_probability, maker_probability)
def fair_price(number_of_contracts, lowest_ask):
    lowest_ask_percentage = lowest_ask / 100
    highest_bid = lowest_ask - 1
    highest_bid_percentage = highest_bid / 100
    taker_odds = 0.07 * lowest_ask_percentage * (1 - lowest_ask_percentage)
    maker_odds = 0.0175 * highest_bid_percentage * (1 - highest_bid_percentage)
    taker_fees = round_up_nearest_cent(taker_odds * number_of_contracts)
    maker_fees = round_up_nearest_cent(maker_odds * number_of_contracts)

    return round(taker_fees + number_of_contracts * lowest_ask_percentage, 2),  round(maker_fees + number_of_contracts * highest_bid_percentage, 2), round(taker_odds + lowest_ask_percentage, 6), round(maker_odds + highest_bid_percentage, 6)

In [3]:
url = "https://api.elections.kalshi.com/trade-api/v2/series/KXNFLGAME"
response = requests.get(url)
series_data = response.json()

print(f"Series Title: {series_data['series']['title']}")
print(f"Frequency: {series_data['series']['frequency']}")
print(f"Category: {series_data['series']['category']}")



Series Title: Professional Football Game
Frequency: custom
Category: Sports


In [3]:
# Get all markets for the KXNFLGAME series
# markets_url = f"https://api.elections.kalshi.com/trade-api/v2/markets?series_ticker=KXNCAAFGAME&status=open"
# markets_response = requests.get(markets_url)
# markets_data = markets_response.json()

markets_data = get_all_markets('KXNCAAFGAME')

# print(f"\nActive markets in KXNFLGAME series:")
# for market in markets_data['markets']:
#     print(f"- {market['ticker']}: {market['title']}")
#     print(f"  Event: {market['event_ticker']}")
#     print(f"  Yes Bid: {market['yes_bid']}¢, Yes Ask: {market['yes_bid']}¢ | Volume: {market['volume']}")
#     print()

# # Get details for a specific event if you have its ticker
# if markets_data['markets']:
#     # Let's get details for the first market's event
#     event_ticker = markets_data['markets'][0]['event_ticker']
#     event_url = f"https://api.elections.kalshi.com/trade-api/v2/events/{event_ticker}"
#     event_response = requests.get(event_url)
#     event_data = event_response.json()

#     print(f"Event Details:")
#     print(f"Title: {event_data['event']['title']}")
#     print(f"Category: {event_data['event']['category']}")

Fetched 100 markets, total: 100
Fetched 100 markets, total: 200
Fetched 100 markets, total: 300
Fetched 100 markets, total: 400
Fetched 100 markets, total: 500
Fetched 100 markets, total: 600
Fetched 100 markets, total: 700


In [65]:
print(markets_data[0])

{'ticker': 'KXNCAAFGAME-25SEP27BYUCOLO-COLO', 'event_ticker': 'KXNCAAFGAME-25SEP27BYUCOLO', 'market_type': 'binary', 'title': 'BYU at Colorado Winner?', 'subtitle': '', 'yes_sub_title': 'Colorado', 'no_sub_title': 'Colorado', 'open_time': '2025-09-11T22:45:00Z', 'close_time': '2025-10-12T02:15:00Z', 'expected_expiration_time': '2025-09-28T05:15:00Z', 'expiration_time': '2025-10-12T02:15:00Z', 'latest_expiration_time': '2025-10-12T02:15:00Z', 'settlement_timer_seconds': 300, 'status': 'active', 'response_price_units': 'usd_cent', 'notional_value': 100, 'notional_value_dollars': '1.0000', 'yes_bid': 27, 'yes_bid_dollars': '0.2700', 'yes_ask': 43, 'yes_ask_dollars': '0.4300', 'no_bid': 57, 'no_bid_dollars': '0.5700', 'no_ask': 73, 'no_ask_dollars': '0.7300', 'last_price': 26, 'last_price_dollars': '0.2600', 'previous_yes_bid': 26, 'previous_yes_bid_dollars': '0.2600', 'previous_yes_ask': 45, 'previous_yes_ask_dollars': '0.4500', 'previous_price': 26, 'previous_price_dollars': '0.2600', 'v

In [ ]:
group_by_title = {}
if type(markets_data) is list:
    markets = markets_data
else:
    markets = markets_data['markets']

for market in markets:
    event_ticker = market['event_ticker']
    date = event_ticker.split('-')[1][:7]
    year , month, day = date[:2], date[2:5], date[5:]
    try:
        d = int(day)
    except ValueError:
        # bad day; skip
        counter += 1
        continue

    # skip unless it's SEP 19–22
    if month != 'SEP' or not (23 <= d <= 29):
        continue 
    title = market['title']
    if title not in group_by_title:
        group_by_title[title] = []
    group_by_title[title].append(market)

for title, markets in reversed(group_by_title.items()):
    print(f"\nMarkets for {title}:")

    market_a = markets[0]
    a_name = market_a['ticker'].split('-')[-1]
        
    try:
        a_yes_bid, a_yes_bid_volume, a_yes_ask, a_yes_ask_volume = get_prices_from_market(market_a)
    except Exception as e:
        print(f"Error getting prices for market {market_a['ticker']}: {e}")
        continue


    market_b = markets[1]
    b_name = market_b['ticker'].split('-')[-1]

    try:
        b_yes_bid, b_yes_bid_volume, b_yes_ask, b_yes_ask_volume = get_prices_from_market(market_b)
    except Exception as e:
        print(f"Error getting prices for market {market_b['ticker']}: {e}")
        continue

    a_lowest_yes_bid = min(a_yes_bid, 100 - b_yes_ask)
    a_lowest_yes_ask = min(a_yes_ask, 100 - b_yes_bid)
    b_lowest_yes_bid = min(b_yes_bid, 100 - a_yes_ask)
    b_lowest_yes_ask = min(b_yes_ask, 100 - a_yes_bid)

    
    a_taker_price, a_maker_price, a_taker_probability, a_maker_probability = fair_price(100, a_lowest_yes_ask)
    b_taker_price, b_maker_price, b_taker_probability, b_maker_probability = fair_price(100, b_lowest_yes_ask)

    # print(f"{a_name}: Lowest price YES Bid: {a_lowest_yes_bid}, YES Ask: {a_lowest_yes_ask}")
    # print(f"{b_name}: Lowest price YES Bid: {b_lowest_yes_bid}, YES Ask: {b_lowest_yes_ask}")

    print(f"{a_name}: Taker Price: ${a_taker_price} ({a_taker_probability}), Maker Price: ${a_maker_price} ({a_maker_probability})")
    print(f"{b_name}: Taker Price: ${b_taker_price} ({b_taker_probability}), Maker Price: ${b_maker_price} ({b_maker_probability})")

    



Markets for Army at East Carolina Winner?:
- ECU: Yes Bid: 65¢ | Volume: 2043, Yes Ask: 66¢ | Volume: 29894
- ARMY: Yes Bid: 34¢ | Volume: 10000, Yes Ask: 36¢ | Volume: 34073
ECU: Taker Price: $67.58 (0.675708), Maker Price: $65.4 (0.653981)
ARMY: Taker Price: $36.6 (0.365925), Maker Price: $34.4 (0.343927)

Markets for Florida St. at Virginia Winner?:
- FSU: Yes Bid: 71¢ | Volume: 59735, Yes Ask: 73¢ | Volume: 81136
- UVA: Yes Bid: 28¢ | Volume: 549, Yes Ask: 29¢ | Volume: 109153
FSU: Taker Price: $73.42 (0.734112), Maker Price: $71.37 (0.713603)
UVA: Taker Price: $30.45 (0.304413), Maker Price: $28.36 (0.283528)

Markets for TCU at Arizona St. Winner?:
- TCU: Yes Bid: 43¢ | Volume: 7379, Yes Ask: 44¢ | Volume: 41337
- ASU: Yes Bid: 55¢ | Volume: 15662, Yes Ask: 56¢ | Volume: 7400
TCU: Taker Price: $45.73 (0.457248), Maker Price: $43.43 (0.434289)
ASU: Taker Price: $57.73 (0.577248), Maker Price: $55.44 (0.554331)

Markets for Houston at Oregon St. Winner?:
- ORST: Yes Bid: 17¢ | Vol

: 

In [8]:
# Get orderbook for a specific market
# Replace with an actual market ticker from the markets list
market_ticker = markets_data['markets'][-1]['ticker']
orderbook_url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{market_ticker}/orderbook"

orderbook_response = requests.get(orderbook_url)
orderbook_data = orderbook_response.json()



print(f"\nOrderbook for {market_ticker}:")

print("\nYES ASKS:")
for bid in orderbook_data['orderbook']['no'][-3:]:  # Show top 5
    print(f"  Price: {100 - bid[0]}¢, Quantity: {bid[1]}")

print("\nYES BIDS:")
for bid in reversed(orderbook_data['orderbook']['yes'][-3:]):  # Show top 5
    print(f"  Price: {bid[0]}¢, Quantity: {bid[1]}")

    
market_ticker = markets_data['markets'][-2]['ticker']
orderbook_url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{market_ticker}/orderbook"

orderbook_response = requests.get(orderbook_url)
orderbook_data = orderbook_response.json()



print(f"\nOrderbook for {market_ticker}:")

print("\nYES ASKS:")
for bid in orderbook_data['orderbook']['no'][-1:]:  # Show top 5
    print(f"  Price: {100 - bid[0]}¢, Quantity: {bid[1]}")

print("\nYES BIDS:")
for bid in reversed(orderbook_data['orderbook']['yes'][-1:]):  # Show top 5
    print(f"  Price: {bid[0]}¢, Quantity: {bid[1]}")


Orderbook for KXNFLGAME-25SEP18MIABUF-BUF:

YES ASKS:
  Price: 90¢, Quantity: 2365471
  Price: 89¢, Quantity: 4451200
  Price: 88¢, Quantity: 5004840

YES BIDS:
  Price: 87¢, Quantity: 363
  Price: 86¢, Quantity: 724921
  Price: 85¢, Quantity: 655602

Orderbook for KXNFLGAME-25SEP18MIABUF-MIA:

YES ASKS:
  Price: 14¢, Quantity: 3339564

YES BIDS:
  Price: 12¢, Quantity: 1589158


In [68]:
print(fair_price(100, 14))
print(fair_price(100, 88))

(14.85, 13.2, 0.148428, 0.131979)
(88.74, 87.2, 0.887392, 0.871979)
